In [1]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time

#-------- CONFIGURAÇÕES DO NAVEGADOR--------#
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": r"C:\Users\Usuario\Downloads\downloadsPV",  # Caminho onde os downloads serão salvos
    "download.prompt_for_download": False,  # Impede o prompt de confirmação de download
    "download.directory_upgrade": True,    # Atualiza automaticamente o diretório de download
    "safebrowsing.enabled": True,          # Habilita o "safe browsing" para evitar alertas de segurança
    "profile.default_content_settings.popups": 0,  # Desabilita popups de download
    "profile.content_settings.exceptions.automatic_downloads": 1  # Permite downloads automáticos
})

# Inicia o navegador com as configurações personalizadas
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico, options=options)
navegador.get("https://acessoainformacao.camaravalparaiso.go.gov.br/cidadao/atos_adm/mp/id=11")
navegador.maximize_window()
time.sleep(3)

# ----------INÍCIO--------------#

# Aceita o botão de cookies, se aparecer
try:
    botao_cookies = WebDriverWait(navegador, 5).until(
        EC.element_to_be_clickable((By.ID, "aceitar_termos"))
    )
    botao_cookies.click()
    print("Cookies aceitos automaticamente.")
except:
    print("Banner de cookies não encontrado ou já aceito.")

time.sleep(3) 

# Loop para processar cada página, ou seja, cada tabela
total_paginas = 16
for pagina in range(1, total_paginas + 1):
    try:
        print(f"Processando a página {pagina}...")
        print("--------------------------------------------------------------------------")

        # Atualiza a lista de linhas antes de iniciar a iteração
        lista_linhas = WebDriverWait(navegador, 30).until(
            EC.presence_of_all_elements_located((By.XPATH, "//table[@class='tb']//tr"))
        )
        print(f"Número de linhas encontradas: {len(lista_linhas)}")

        # Loop para percorrer cada linha da tabela
        cont_qtd_iteracoes = 0
        for linha_index in range(1, len(lista_linhas)):
            try:
                # Atualiza a lista de linhas antes de interagir (Garante que os elementos estão disponíveis)
                lista_linhas = WebDriverWait(navegador, 30).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//table[@class='tb']//tr"))
                )

                # Obtém a linha específica após atualizar a lista
                linha = lista_linhas[linha_index]

                # Faz a rolagem até a linha
                actions = ActionChains(navegador)
                actions.move_to_element(linha).perform()
                time.sleep(1) 

                # Clica na linha
                WebDriverWait(navegador, 30).until(EC.element_to_be_clickable(linha)).click()
                cont_qtd_iteracoes += 1

                # Pausa para carregar a página de download
                time.sleep(3)

                # Espera o botão de download ficar clicável e realiza o download
                try:
                    botao_download = WebDriverWait(navegador, 30).until(
                        EC.element_to_be_clickable((By.ID, "download_anexos"))
                    )
                    botao_download.click()
                    print(f"Download realizado com sucesso da linha {linha_index} na página {pagina}.")
                except Exception as e:
                    print(f"Erro ao clicar no botão de download da linha {linha_index} na página {pagina}: {e}")

                # Pausa antes de clicar no botão de voltar
                time.sleep(3)

                # Clica no botão de voltar
                botao_voltar = WebDriverWait(navegador, 30).until(
                    EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/main/div[2]/div[2]/div[2]/div[2]/button"))
                )
                botao_voltar.click()

                # Pausa para voltar à tabela
                time.sleep(3)

                # Verifica se ainda está na página correta após voltar
                pagina_atual = WebDriverWait(navegador, 10).until(
                    EC.presence_of_element_located((By.NAME, "ir_para"))
                ).get_attribute("value")

                if int(pagina_atual) != pagina:
                    print(f"⚠ Página incorreta detectada! Voltando para {pagina}...")
                    select_element = WebDriverWait(navegador, 30).until(
                        EC.presence_of_element_located((By.NAME, "ir_para"))
                    )
                    select = Select(select_element)
                    select.select_by_value(str(pagina))  # Seleciona a página correta

                    # Aguarda o recarregamento da tabela
                    WebDriverWait(navegador, 30).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//table[@class='tb']//tr"))
                    )
                    time.sleep(3)  # Pausa extra para estabilidade

                print(f"✅ Página {pagina} confirmada. Continuando...")

            except Exception as e:
                print(f"Erro ao processar a linha {linha_index} na página {pagina}: {e}")

            # Passa para a próxima página quando todas as 15 linhas forem processadas
            if cont_qtd_iteracoes == 15:
                print("--------------------------------------------------------------------------")
                print(f"Indo para a próxima página ({pagina + 1})...")
                # Verificar o método de troca de página!!!!!!!!!!!!!!!!!!!
                # Pausa
                time.sleep(5)

                try:
                    # Encontra o elemento select pelo name
                    select_element = WebDriverWait(navegador, 30).until(
                        EC.presence_of_element_located((By.NAME, "ir_para")) 
                    )

                    # Cria um objeto Select
                    select = Select(select_element)

                    # Converte página para string (pois opções do select são strings)
                    pagina_str = str(pagina + 1)

                    # Seleciona a página desejada
                    select.select_by_value(pagina_str)

                    print(f"Selecionando a página {pagina + 1}...")

                    # Atualiza a lista de linhas antes de interagir (Garante que os elementos estão disponíveis)
                    lista_linhas = WebDriverWait(navegador, 30).until(
                        EC.presence_of_all_elements_located((By.XPATH, "//table[@class='tb']//tr"))
                    )

                    # Pausa para garantir que a tabela carregou
                    time.sleep(5)  

                except Exception as e:
                    print(f"Erro ao selecionar a página {pagina + 1}: {e}")

    except Exception as e:
        print(f"Erro ao processar a página {pagina}: {e}")

print("Processamento completo!")



Cookies aceitos automaticamente.
Processando a página 1...
--------------------------------------------------------------------------
Número de linhas encontradas: 16
Download realizado com sucesso da linha 1 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 2 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 3 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 4 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 5 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 6 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 7 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 8 na página 1.
✅ Página 1 confirmada. Continuando...
Download realizado com sucesso da linha 9 na página 1.
✅ Página 1 confirmada. Continuando

In [1]:
import os

caminho = r"C:\Users\Usuario\Downloads\downloadsPV"
arquivos = [f for f in os.listdir(caminho) if os.path.isfile(os.path.join(caminho, f))]
print(f"Quantidade de arquivos: {len(arquivos)}")


Quantidade de arquivos: 227
